In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import json

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

In [ ]:
df_raw = pd.read_csv("dados/inputs/microdados_ed_basica_2024.csv", sep=";", encoding="latin1")

print(df_raw.shape)

# é  um df muito grande (215_545, 426), primeiro passo: filtrar as colunas

## Filtrando Colunas

#### Atualizando o BD

In [ ]:
# df_filtro = pd.read_excel("dados/temporarios/filtro_colunas.xlsx", skiprows=8, usecols=[1, 2])

# df_filtro = df_filtro[~df_filtro["util"].isin(["DESCARTAR", "TALVEZ", "AUSENTE", np.nan])]

# lista_cols = df_filtro["Nome da Variável"].astype(str).to_list()

# print(f"Reduzimos para {len(lista_cols)} colunas")

# with open("dados/banco_dados/colunas_relevantes_md_edb.json", "w", encoding="utf-8") as f:
#     json.dump(lista_cols, f, ensure_ascii=False, indent=4)
    

#### Buscando nomes colunas relevantes no BD

In [ ]:

with open("dados/banco_dados/colunas_relevantes_md_edb.json", "r", encoding="utf-8") as f:
    cols_interesse = json.load(f)

df_cols_filtrado = df_raw[cols_interesse]

df_cols_filtrado.shape  # Reduzimos para 63 colunas


## Combinando Colunas

In [ ]:
def soma_colunas(df: pd.DataFrame, colunas: list[str], novo_nome: str, drop = True):
    """
    df: DataFrame
    colunas: lista com nomes de colunas a serem somadas
    novo_nome: nome da nova coluna criada
    """
    df[novo_nome] = df[colunas].sum(axis=1).astype(int)
    if drop:
        df = df.drop(columns=colunas)
        
    return df



In [ ]:

df = df_cols_filtrado.copy()


infraestrutura = ["IN_ESGOTO_REDE_PUBLICA", "IN_ENERGIA_REDE_PUBLICA",
                  "IN_LIXO_SERVICO_COLETA"]
df = soma_colunas(df, infraestrutura, "infraestrutura")



estrutura_pobre = ["IN_TERREIRAO", "IN_ACESSIBILIDADE_INEXISTENTE"]
df = soma_colunas(df, estrutura_pobre, "estrutura_pobre")

estrutura_basica = ["IN_ALMOXARIFADO", "IN_BIBLIOTECA", "IN_LABORATORIO_INFORMATICA",
                    "IN_PATIO_DESCOBERTO", "IN_QUADRA_ESPORTES_DESCOBERTA"]
df = soma_colunas(df, estrutura_basica, "estrutura_basica")

estrutura_padrao = ["IN_AUDITORIO", "IN_BIBLIOTECA_SALA_LEITURA",
                    "IN_LABORATORIO_CIENCIAS", "IN_PATIO_COBERTO"]
df = soma_colunas(df, estrutura_padrao, "estrutura_padrao")

estrutura_premium = ["IN_AREA_PLANTIO", "IN_BANHEIRO_CHUVEIRO", "IN_PISCINA", "IN_SALA_ATELIE_ARTES",
                     "IN_SALA_MUSICA_CORAL", "IN_SALA_ESTUDIO_DANCA", "IN_SALA_ESTUDIO_GRAVACAO",
                     "IN_SALA_REPOUSO_ALUNO", "IN_ACESSIBILIDADE_ELEVADOR", "IN_MATERIAL_PED_MUSICAL"]
df = soma_colunas(df, estrutura_premium, "estrutura_premium")



ativos_basico = ["QT_EQUIP_TV", "QT_EQUIP_MULTIMIDIA", "QT_DESKTOP_ALUNO"]
df = soma_colunas(df, ativos_basico, "qt_ativos_basico")

ativos_premium = ["QT_EQUIP_LOUSA_DIGITAL", "QT_COMP_PORTATIL_ALUNO", "QT_TABLET_ALUNO"]
df = soma_colunas(df, ativos_premium, "qt_ativos_premium")



pessoal_basico = ["QT_PROF_ADMINISTRATIVOS", "QT_PROF_SERVICOS_GERAIS",
                  "QT_PROF_BIBLIOTECARIO", "QT_PROF_ALIMENTACAO", "QT_PROF_SECRETARIO"]
df = soma_colunas(df, pessoal_basico, "pessoal_basico")

pessoal_padrao = ["QT_PROF_SAUDE", "QT_PROF_COORDENADOR", "QT_PROF_PEDAGOGIA", "QT_PROF_MONITORES",
                "QT_PROF_GESTAO", "QT_PROF_ASSIST_SOCIAL"]
df = soma_colunas(df, pessoal_padrao, "pessoal_padrao")

pessoal_premium = ["QT_PROF_FONAUDIOLOGO", "QT_PROF_NUTRICIONISTA", "QT_PROF_PSICOLOGO",
               "QT_PROF_SEGURANCA", "QT_PROF_AGRICOLA"]
df = soma_colunas(df, pessoal_premium, "pessoal_premium")



alunos = ["QT_MAT_INF", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF", "QT_MAT_MED"]
df = soma_colunas(df, alunos, "qt_alunos", drop=False)


professores = ["QT_DOC_INF", "QT_DOC_FUND_AI", "QT_DOC_FUND_AF", "QT_DOC_MED"]
df = soma_colunas(df, professores, "qt_professores")


#### Arrumando coluna TP_OCUPACAO_PREDIO_ESCOLAR

In [ ]:
df["tipo_ocupacao"] = (df["TP_OCUPACAO_PREDIO_ESCOLAR"]
                                 .fillna(2)
                                 .replace({1:3, 2:1, 3:2}))  # 1 alugado, 2 cedido, 3 proprio


df_cols_combinado = df.drop(columns=["TP_OCUPACAO_PREDIO_ESCOLAR"])


df_cols_combinado.shape  # reduzimos para 25 colunas

## Filtrando linhas

In [ ]:
df = df_cols_combinado.copy()

df = df[df["TP_DEPENDENCIA"] == 4]  # Mantem escolas particulares, reduz para 52_568
df = df[(df["TP_CATEGORIA_ESCOLA_PRIVADA"] != 3)]  # Remove escolas particulares confessionais, reduz para 52_101
df = df[(df["TP_SITUACAO_FUNCIONAMENTO"] == 1)]  # Mantem escolas em atividade, reduz para 42_751
df = df[df["IN_MEDIACAO_PRESENCIAL"] == 1]  # Remove escolas que nao tem aula presencial, reduz para 42_470

df = df.drop(columns=["TP_DEPENDENCIA", "TP_CATEGORIA_ESCOLA_PRIVADA", "TP_SITUACAO_FUNCIONAMENTO", "IN_MEDIACAO_PRESENCIAL"])


df_linhas_filtrado = df[df["qt_alunos"] > 20] # Remove escolas com menos de 20 alunos, reduz para 37_178


df_linhas_filtrado.shape  # reduzimos para 37_178 linhas

## Tratando Outliers

In [ ]:
def tratar_outliers(serie, x, achatar=False):
    """
    serie: pandas Series numérica
    x: percentil (0–100)
    achatar: se True, substitui valores maiores pelo teto
    """
    
    limite = serie.quantile(x/100)
    mediana = serie.median()
    
    # substitui os valores acima do percentil pela mediana
    tratada = serie.copy()
    if achatar:
        tratada[tratada > limite] = round(limite)
    else:
        tratada[tratada > limite] = round(mediana)
    
    return tratada


### Investigar distribuição dos valores

In [ ]:
col = "qt_alunos"

df_b = df_linhas_filtrado.copy()
df_b[col] = tratar_outliers(df_b[col], 97, True)

fig = px.histogram(
    df_b,
    x=col,
    # nbins=70
)
fig.update_layout(
    bargap=0.05,
    width=800,
    height=500
)
fig.show()

# display(df_linhas_filtrado[df_linhas_filtrado[col] == 2092])

print(df_b[col].sort_values(ascending=False).value_counts(sort=False))

### Aplicando alterações

In [ ]:
df = df_linhas_filtrado.copy()

# df_corr["alunos_p_professor"] = df_corr["qt_alunos"] / df_corr["qt_professores"]
# df_corr["nota_enem"] = df_corr["nota_objetiva"] + df_corr["nota_redacao"]

# df_corr = df_corr.drop(columns=["qt_alunos", "qt_professores",
#                                 "nota_objetiva", "nota_redacao"
#                                 ])


df["QT_SALAS_UTILIZADAS"] = tratar_outliers(df["QT_SALAS_UTILIZADAS"], 98)
df["IN_EXAME_SELECAO"] = df["IN_EXAME_SELECAO"].replace({9:0})  # provavelmente esse 9 era um 0 feio
df["QT_MAT_INF"] = tratar_outliers(df["QT_MAT_INF"], 99)
df["QT_MAT_FUND_AI"] = tratar_outliers(df["QT_MAT_FUND_AI"], 99, True)
df["QT_MAT_FUND_AF"] = tratar_outliers(df["QT_MAT_FUND_AF"], 99, True)
df["QT_MAT_MED"] = tratar_outliers(df["QT_MAT_MED"], 99, True)

# sem outlier: infraestrutura, estrutura_pobre, estrutura_basica, estrutura_padrao, estrutura_premium

df["qt_ativos_basico"] = tratar_outliers(df["qt_ativos_basico"], 99, True)
df["qt_ativos_premium"] = tratar_outliers(df["qt_ativos_premium"], 98, True)
df["pessoal_basico"] = tratar_outliers(df["pessoal_basico"], 99, True)
df["pessoal_padrao"] = tratar_outliers(df["pessoal_padrao"], 96, True)
df["pessoal_premium"] = tratar_outliers(df["pessoal_premium"], 99.3, True)
df["qt_alunos"] = tratar_outliers(df["qt_alunos"], 97, True)
df["qt_professores"] = tratar_outliers(df["qt_professores"], 99, True)

df["alunos_p_professor"] = df["qt_alunos"] / df["qt_professores"]
df["alunos_p_professor"] = tratar_outliers(df["alunos_p_professor"], 99, True)

df["alunos_p_sala"] = df["qt_alunos"] / df["QT_SALAS_UTILIZADAS"]
df["alunos_p_sala"] = tratar_outliers(df["alunos_p_sala"], 98, True)

# sem outlier: tipo_ocupacao

df_corr = df

print(df_corr.shape)  # 37_178 linhas, 23 colunas
display(df_corr.head())

## Salvando resultados

In [ ]:
df_corr.to_csv("dados/temporarios/01-md_edb_tratado.csv", index=False)